## Mẫu sử dụng llm tạo response

In [ ]:
from llm import llm, create_llm_prompt

# model_name="gpt-5-2025-08-07" #gpt 5
model_name="gpt-5-mini-2025-08-07" #gpt 5 mini
# model_name="local"
# model_name="gemini-2.5-flash-lite"

my_llm = llm(model_name=model_name)


EMPTY
Using API Key from: gemini-2.5-flash-lite_API_KEY
Using Base URL from: gemini-2.5-flash-lite_BASE_URL


In [2]:
prompt=create_llm_prompt(False,{"instruction":"you are helpful assistant",
                               "response_structure":"""{"answer":str}""",
                               "input":"1+1="} )

In [3]:
response=my_llm.generate(prompt, 200, True, 1.25, 10)

Total cost of the API call: $0.000171


## Model qwen3 14B

In [8]:
from llm import llm, create_llm_prompt

# model_name="gpt-5-2025-08-07" #gpt 5
# model_name="gpt-5-mini-2025-08-07" #gpt 5 mini
model_name="local"
# model_name="gemini-2.5-flash-lite"

my_llm = llm(model_name=model_name)


Using API Key from: local_API_KEY
Using Base URL from: local_BASE_URL


In [9]:
import pandas as pd
test_case_df=pd.read_csv("dataset/work_rule_results.csv", encoding="utf-8")

In [10]:
from llm import create_llm_prompt
from shared.common_fn import run_parallel_tasks_with_saving
from prompts import CREATE_EXPLANATION_LABEL_TEMPLATE
import copy
from json_repair import repair_json 
import json
import time
def create_qwen_response_parallel(
    test_case_df,
    output_csv_path="model_response/qwen3_14B_response.csv",
    max_workers=5,
    save_every=10,
):
    """Sinh nội quy lao động từ điều luật bằng LLM, chạy song song."""
    
    # Chuẩn bị danh sách job
    jobs = [
        (i, test_case_df.iloc[i])
        for i in range(test_case_df.shape[0])
    ]

    # Định nghĩa hàm xử lý từng job
    def job_fn(idx, row):
        try:
            template = copy.deepcopy(CREATE_EXPLANATION_LABEL_TEMPLATE)
            template["input"] = template["input"].format(related_law=row["law"], work_rule=row["work_rule_ja"])
            prompt = create_llm_prompt(is_model_qwen=True, template=template)
            start_generation_time = time.time()
            response = my_llm.generate(prompt, max_tokens=4000)
            end_generation_time = time.time()
            generation_time=end_generation_time - start_generation_time
            print(f"⏱️ Generation time for job {idx}: {generation_time} seconds")
            price = 0
            response=response.choices[0].text
            response_json = repair_json(response, ensure_ascii=False)
            parsed = json.loads(response_json)

            return {
                "law": row["law"],
                "label": row["label"],
                "work_rule_ja": row["work_rule_ja"],
                "explanation_ja": row["explanation_ja"],
                "generated_label": parsed["label"],
                "generated_explanation": parsed["explanation_ja"],
                "price": price,
                "generation_time": generation_time
            }

        except Exception as e:
            print(f"⚠️ Error in job {idx}: {e}")
            return {
                "law": row["law"],
                "label": row["label"],
                "work_rule_ja": row["work_rule_ja"],
                "explanation_ja": row["explanation_ja"],
                "generated_label": "",
                "generated_explanation": "",
                "price": 0
            }

    # Gọi hàm tổng quát
    results_df = run_parallel_tasks_with_saving(
        jobs=jobs,
        job_fn=job_fn,
        output_csv_path=output_csv_path,
        columns=["vi_law", "converted_law_ja", "price"],
        max_workers=max_workers,
        save_every=save_every,
    )

    return results_df

In [11]:
qwen_response_results=create_qwen_response_parallel(
    test_case_df)

🚀 Running 160 jobs with 5 workers...
/home/trungquang/llm_models/qwen/qwen3-14b-awq
/home/trungquang/llm_models/qwen/qwen3-14b-awq
/home/trungquang/llm_models/qwen/qwen3-14b-awq
/home/trungquang/llm_models/qwen/qwen3-14b-awq
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 1: 1.2285096645355225 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 0: 1.4706008434295654 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 3: 1.6022725105285645 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 4: 1.6430211067199707 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 2: 1.866816759109497 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 8: 0.9082906246185303 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 5: 1.3374450206756592 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱

/home/thanhnguyen/code/compare_model_response_in_legal_domain/shared/common_fn.py:46: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return df[price_column].fillna(0).sum()


⏱️ Generation time for job 12: 0.7982807159423828 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 10: 1.7150678634643555 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 11: 1.9708549976348877 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 13: 1.2085249423980713 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 14: 1.315922498703003 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 15: 1.379324197769165 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 18: 0.8920507431030273 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 19: 1.2411398887634277 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 16: 1.80552339553833 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 17: 1.5504059791564941 seconds
/

/home/thanhnguyen/code/compare_model_response_in_legal_domain/shared/common_fn.py:46: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return df[price_column].fillna(0).sum()


⏱️ Generation time for job 20: 1.3543305397033691 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 21: 1.381666898727417 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 22: 1.3107893466949463 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 24: 1.8549537658691406 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 23: 1.8936121463775635 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 25: 1.2853524684906006 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 26: 1.427917718887329 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 27: 1.2470042705535889 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 28: 1.0247137546539307 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 31: 1.0347316265106201 seconds

...

/home/thanhnguyen/code/compare_model_response_in_legal_domain/shared/common_fn.py:46: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return df[price_column].fillna(0).sum()


⏱️ Generation time for job 140: 1.604463815689087 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 141: 1.8107354640960693 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 144: 1.4167184829711914 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 142: 1.6049425601959229 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 143: 1.6834192276000977 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 145: 1.4341230392456055 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 146: 1.2794075012207031 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 148: 1.6070284843444824 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 147: 2.495068073272705 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 149: 2.54243254661560

/home/thanhnguyen/code/compare_model_response_in_legal_domain/shared/common_fn.py:46: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return df[price_column].fillna(0).sum()


⏱️ Generation time for job 151: 1.6646223068237305 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 150: 2.186244487762451 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 152: 2.0300614833831787 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 154: 1.0467376708984375 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 153: 1.5334515571594238 seconds
/home/trungquang/llm_models/qwen/qwen3-14b-awq
⏱️ Generation time for job 155: 1.434570550918579 seconds
⏱️ Generation time for job 156: 1.380526065826416 seconds
⏱️ Generation time for job 157: 1.7290115356445312 seconds
⏱️ Generation time for job 159: 1.4979090690612793 seconds
⏱️ Generation time for job 158: 2.2010231018066406 seconds
💾 Saved 160 rows (processed 160/160) | 💰 Total = 0.0000
✅ Completed. Total rows: 160 saved to model_response/qwen3_14B_response.csv


/home/thanhnguyen/code/compare_model_response_in_legal_domain/shared/common_fn.py:46: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return df[price_column].fillna(0).sum()


## Model gemini flash 2.5

In [17]:
from llm import llm

# model_name="gpt-5-2025-08-07" #gpt 5
# model_name="gpt-5-mini-2025-08-07" #gpt 5 mini
# model_name="local"
model_name="gemini-2.5-flash-lite"

my_llm = llm(model_name=model_name)

Using API Key from: gemini-2.5-flash-lite_API_KEY
Using Base URL from: gemini-2.5-flash-lite_BASE_URL


In [18]:
import pandas as pd
test_case_df=pd.read_csv("dataset/work_rule_results.csv", encoding="utf-8")

In [19]:
from llm import create_llm_prompt
from shared.common_fn import run_parallel_tasks_with_saving
from prompts import CREATE_EXPLANATION_LABEL_TEMPLATE
import copy
from json_repair import repair_json 
import json
def create_gemini_response_parallel(
    test_case_df,
    output_csv_path="model_response/gemini_2.5_flash_lite_response.csv",
    max_workers=5,
    save_every=10,
):
    """Sinh nội quy lao động từ điều luật bằng LLM, chạy song song."""
    
    # Chuẩn bị danh sách job
    jobs = [
        (i, test_case_df.iloc[i])
        for i in range(test_case_df.shape[0])
    ]

    # Định nghĩa hàm xử lý từng job
    def job_fn(idx, row):
        try:
            template = copy.deepcopy(CREATE_EXPLANATION_LABEL_TEMPLATE)
            template["input"] = template["input"].format(related_law=row["law"], work_rule=row["work_rule_ja"])
            prompt = create_llm_prompt(is_model_qwen=True, template=template)

            start_generation_time = time.time()
            response = my_llm.generate(prompt, max_tokens=4000)
            end_generation_time = time.time()
            generation_time=end_generation_time - start_generation_time
            print(f"⏱️ Generation time for job {idx}: {generation_time} seconds")
            price = my_llm.calculate_price(response)
            response_json = repair_json(response.text, ensure_ascii=False)
            parsed = json.loads(response_json)

            return {
                "law": row["law"],
                "label": row["label"],
                "work_rule_ja": row["work_rule_ja"],
                "explanation_ja": row["explanation_ja"],
                "generated_label": parsed["label"],
                "generated_explanation": parsed["explanation_ja"],
                "price": price,
                "generation_time": generation_time
            }

        except Exception as e:
            print(f"⚠️ Error in job {idx}: {e}")
            return {
                "law": row["law"],
                "label": row["label"],
                "work_rule_ja": row["work_rule_ja"],
                "explanation_ja": row["explanation_ja"],
                "generated_label": "",
                "generated_explanation": "",
                "price": 0
            }

    # Gọi hàm tổng quát
    results_df = run_parallel_tasks_with_saving(
        jobs=jobs,
        job_fn=job_fn,
        output_csv_path=output_csv_path,
        columns=["law", "label","work_rule_ja","explanation_ja","generated_label","generated_explanation","price"],
        max_workers=max_workers,
        save_every=save_every,
    )

    return results_df

In [20]:
gemini_response_results=create_gemini_response_parallel(
    test_case_df)

🚀 Running 160 jobs with 5 workers...
gemini-2.5-flash-lite
gemini-2.5-flash-lite
gemini-2.5-flash-lite
gemini-2.5-flash-lite
gemini-2.5-flash-lite
⏱️ Generation time for job 1: 1.1489794254302979 seconds
gemini-2.5-flash-lite
⏱️ Generation time for job 2: 1.2285752296447754 seconds
gemini-2.5-flash-lite
⏱️ Generation time for job 4: 1.3083455562591553 seconds
gemini-2.5-flash-lite
⏱️ Generation time for job 0: 1.3283774852752686 seconds
gemini-2.5-flash-lite
⏱️ Generation time for job 3: 1.3934578895568848 seconds
gemini-2.5-flash-lite
⏱️ Generation time for job 5: 0.9240589141845703 seconds
gemini-2.5-flash-lite
⏱️ Generation time for job 8: 1.2603955268859863 seconds
gemini-2.5-flash-lite
⏱️ Generation time for job 7: 1.3418910503387451 seconds
gemini-2.5-flash-lite
⏱️ Generation time for job 9: 1.2689855098724365 seconds
gemini-2.5-flash-lite
⏱️ Generation time for job 6: 1.4415521621704102 seconds
gemini-2.5-flash-lite
💾 Saved 10 rows (processed 10/160) | 💰 Total = 0.0039


/home/thanhnguyen/code/compare_model_response_in_legal_domain/shared/common_fn.py:61: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, temp_df], ignore_index=True)


⏱️ Generation time for job 10: 1.1865811347961426 seconds
gemini-2.5-flash-lite
⏱️ Generation time for job 12: 0.8569450378417969 seconds
gemini-2.5-flash-lite
⏱️ Generation time for job 14: 1.0095152854919434 seconds
gemini-2.5-flash-lite
⏱️ Generation time for job 13: 1.1799345016479492 seconds
gemini-2.5-flash-lite
⏱️ Generation time for job 11: 1.3528168201446533 seconds
gemini-2.5-flash-lite
⏱️ Generation time for job 15: 1.2667782306671143 seconds
gemini-2.5-flash-lite
⏱️ Generation time for job 16: 1.2586216926574707 seconds
gemini-2.5-flash-lite
⏱️ Generation time for job 18: 1.0971999168395996 seconds
gemini-2.5-flash-lite
⏱️ Generation time for job 19: 1.0117359161376953 seconds
gemini-2.5-flash-lite
⏱️ Generation time for job 17: 1.5962908267974854 seconds
gemini-2.5-flash-lite
💾 Saved 20 rows (processed 20/160) | 💰 Total = 0.0082
⏱️ Generation time for job 20: 0.9263274669647217 seconds
gemini-2.5-flash-lite
⏱️ Generation time for job 22: 0.7661151885986328 seconds
gemini-2

## Model gpt 5 mini

In [2]:
from llm import llm

# model_name="gpt-5-2025-08-07" #gpt 5
model_name="gpt-5-mini-2025-08-07" #gpt 5 mini
# model_name="local"
# model_name="gemini-2.5-flash-lite"

my_llm = llm(model_name=model_name)

Using API Key from: gpt-5-mini-2025-08-07_API_KEY
Using Base URL from: gpt-5-mini-2025-08-07_BASE_URL


In [3]:
import pandas as pd
test_case_df=pd.read_csv("dataset/work_rule_results.csv", encoding="utf-8")

In [6]:
from llm import create_llm_prompt
from shared.common_fn import run_parallel_tasks_with_saving
from prompts import CREATE_EXPLANATION_LABEL_TEMPLATE
import copy
from json_repair import repair_json 
import json
import time
def create_gpt_response_parallel(
    test_case_df,
    output_csv_path="model_response/chatgpt_5_mini_response.csv",
    max_workers=5,
    save_every=10,
):
    """Sinh nội quy lao động từ điều luật bằng LLM, chạy song song."""
    
    # Chuẩn bị danh sách job
    jobs = [
        (i, test_case_df.iloc[i])
        for i in range(test_case_df.shape[0])
    ]

    # Định nghĩa hàm xử lý từng job
    def job_fn(idx, row):
        try:
            template = copy.deepcopy(CREATE_EXPLANATION_LABEL_TEMPLATE)
            template["input"] = template["input"].format(related_law=row["law"], work_rule=row["work_rule_ja"])
            prompt = create_llm_prompt(is_model_qwen=True, template=template)

            start_generation_time = time.time()
            response = my_llm.generate(prompt, max_tokens=4000)
            end_generation_time = time.time()
            generation_time=end_generation_time - start_generation_time
            print(f"⏱️ Generation time for job {idx}: {generation_time} seconds")
            price = my_llm.calculate_price(response)
            response_json = repair_json(response.output_text, ensure_ascii=False)
            parsed = json.loads(response_json)

            return {
                "law": row["law"],
                "label": row["label"],
                "work_rule_ja": row["work_rule_ja"],
                "explanation_ja": row["explanation_ja"],
                "generated_label": parsed["label"],
                "generated_explanation": parsed["explanation_ja"],
                "price": price,
                "generation_time": generation_time
            }

        except Exception as e:
            print(f"⚠️ Error in job {idx}: {e}")
            return {
                "law": row["law"],
                "label": row["label"],
                "work_rule_ja": row["work_rule_ja"],
                "explanation_ja": row["explanation_ja"],
                "generated_label": "",
                "generated_explanation": "",
                "price": 0
            }

    # Gọi hàm tổng quát
    results_df = run_parallel_tasks_with_saving(
        jobs=jobs,
        job_fn=job_fn,
        output_csv_path=output_csv_path,
        columns=["law", "label","work_rule_ja","explanation_ja","generated_label","generated_explanation","price"],
        max_workers=max_workers,
        save_every=save_every,
    )

    return results_df

In [7]:
results_df=create_gpt_response_parallel(test_case_df)

📂 Loaded 160 existing rows from model_response/chatgpt_5_mini_response.csv
🚀 Running 160 jobs with 5 workers...
gpt-5-mini-2025-08-07
gpt-5-mini-2025-08-07
gpt-5-mini-2025-08-07
gpt-5-mini-2025-08-07
gpt-5-mini-2025-08-07
⏱️ Generation time for job 0: 6.765291213989258 seconds
gpt-5-mini-2025-08-07
⏱️ Generation time for job 2: 8.121243000030518 seconds
gpt-5-mini-2025-08-07
⏱️ Generation time for job 3: 8.53188681602478 seconds
gpt-5-mini-2025-08-07
⏱️ Generation time for job 4: 8.654658555984497 seconds
gpt-5-mini-2025-08-07
⏱️ Generation time for job 1: 11.438015460968018 seconds
gpt-5-mini-2025-08-07
⏱️ Generation time for job 5: 4.830863952636719 seconds
gpt-5-mini-2025-08-07
⏱️ Generation time for job 8: 5.16118049621582 seconds
gpt-5-mini-2025-08-07
⏱️ Generation time for job 6: 8.220705270767212 seconds
gpt-5-mini-2025-08-07
⏱️ Generation time for job 7: 8.887384176254272 seconds
gpt-5-mini-2025-08-07
⏱️ Generation time for job 9: 8.81313681602478 seconds
gpt-5-mini-2025-08-07
